In [ ]:
import numpy as np

import keras, os

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt


### Data prep

In [ ]:
# TODO: data augmentation ?

In [ ]:
# Notes: I assume 
# - you have objects train_images and train_labels (and corresponding test_*) with dimentions: 
#   train_images.shape = (n examples, px width, px height)
#   train_labels.shape = (n examples, )
# - pixel values are NORMALISED --> smaller values work better ususally
# - We have two classes that are more or less balanced 

In [ ]:
# Reshape images to 3D for keras imput (if you have not already) 
train_images = np.expand_dims(train_images, axis=3)
# do the same for test_images 

train_labels = tf.keras.utils.to_categorical(train_labels) # just turns vector labels to a 1-hot array, not sure if you need it but probably 
# same for test_labels

_, img_dim1, img_dim2, img_dim3 = train_images.shape

### Build the model 

In [ ]:
# Simple CNN model 

# Hyperparameters (PLAY KIDS, PLAY)
num_filters = 8 
filter_size = 3
pool_size = 2

activation_inner, activation_outer = 'relu', 'sigmoid' # 'softmax'

# Sequential stacks the layers together 
cnn_model = tf.keras.Sequential([
    
    # First Convolution and Pooling Layers
    tf.keras.layers.Conv2D(num_filters, filter_size, 
                           input_shape=(img_dim1, img_dim2, img_dim3),
                           activation=activation_inner),
    tf.keras.layers.MaxPooling2D(pool_size=pool_size),
    # tf.keras.layers.BatchNormalization()
    
    # Add second convolution and pooling Layers (try without first)
    # tf.keras.layers.Conv2D(num_filters2, filter_size2, activation=activation_inner),
    # tf.keras.layers.MaxPooling2D(pool_size=pool_size2), 

    # Dropout example
    tf.keras.layers.Dropout(0.2), # Nice for preventing overfitting 

    tf.keras.layers.Flatten(), 
    # Can add more Dense layers here too often done
    tf.keras.layers.Dense(1, activation=activation_outer), # 2 categories no? 
])

cnn_model.summary()
#tf.keras.utils.plot_model(cnn_model, show_shapes=True)

### Compile 

In [ ]:
optimizer = 'adam' # this is usually quite good, I would not touch for now 
loss_func = 'binary_crossentropy' # we have two categories no..? or 'categorical_crossentropy'
# try also tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) for sparse labels
metrics = ['accuracy'] # Can use F1 if you have very unbalanced classes, not sure 

cnn_model.compile(
    optimizer, 
    loss = loss_func,
    metrics=metrics
)

### Train it

In [ ]:
n_epochs = 100 # Iterations, keep this low if you are overfitting, or you can try to rely on the callbacks I define below

# Define some to prevent overfitting callbacks
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "cnn_stocazzo.best_epoch", save_best_only=True
)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)


history = cnn_model.fit(
    train_images, 
    train_labels, # just turns vector labels to a 1-hot array, not sure if you need it 
    epochs = n_epochs, 
    validation_data=(test_images, test_labels),
    callbacks=[checkpoint_cb, early_stopping_cb],
    verbose=2,
)

### Evaluate

In [ ]:
# I plot accuracy you can plot loss too if helpful 
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label = 'test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

test_loss, test_acc = cnn_model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

fig, ax = plt.subplots(2, 1, figsize=(20, 10))
ax = ax.ravel()

### Save

In [ ]:
cnn_model.save_weights('cnn_stocazzo.h5')